In [1]:
from loaders.EitzDataLoader import EitzDataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from models.models import SqueezeNet

from __future__ import print_function, division
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import os
import copy

In [2]:
class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)
        
args = {
    "num_labels": 250
}

args = Struct(**args)

In [3]:
model = SqueezeNet(args)
model.type(torch.cuda.FloatTensor)

/home/shared/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
/home/shared/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.


SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0

In [4]:
dataloaders = {'train': EitzDataLoader(4, 16, 'train'), 
               'val': EitzDataLoader(4, 16, 'val'), 
               'test': EitzDataLoader(4, 16, 'test')}

In [5]:
device = 'cuda'

In [6]:
dataset_sizes = {'train': 15000,
                'val': 2500,
                'test': 2500}

In [16]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.float()
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    print(preds)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_loss > best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [31]:
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=.0001, weight_decay=1e-2)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=.5)

In [32]:
best_model = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.5124 Acc: 0.0000
val Loss: 1.9878 Acc: 0.0000

Epoch 1/9
----------
train Loss: 0.4637 Acc: 0.0000
val Loss: 1.9443 Acc: 0.0000

Epoch 2/9
----------
train Loss: 0.4237 Acc: 0.0000
val Loss: 1.9289 Acc: 0.0000

Epoch 3/9
----------
train Loss: 0.3941 Acc: 0.0000
val Loss: 1.8165 Acc: 0.0000

Epoch 4/9
----------
train Loss: 0.3624 Acc: 0.0000
val Loss: 1.9974 Acc: 0.0000

Epoch 5/9
----------
train Loss: 0.3430 Acc: 0.0000
val Loss: 1.9393 Acc: 0.0000

Epoch 6/9
----------
train Loss: 0.3337 Acc: 0.0000
val Loss: 2.1142 Acc: 0.0000

Epoch 7/9
----------
train Loss: 0.3149 Acc: 0.0000
val Loss: 2.0105 Acc: 0.0000

Epoch 8/9
----------
train Loss: 0.1100 Acc: 0.0000
val Loss: 2.0749 Acc: 0.0000

Epoch 9/9
----------
train Loss: 0.1052 Acc: 0.0000
val Loss: 2.0755 Acc: 0.0000

Training complete in 39m 8s
Best val loss:  inf


In [34]:
torch.save(best_model.state_dict(), f'/home/robincheong/sbir/checkpoints/SqueezeNet/eitz2012/attempt2')